In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
!pip install transformers torch
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelWithLMHead
import time

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.31.0 which is incompatible.


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
device_map = {"": 0}
model_name='Dan2205/Llama-2-7b-chat-finetune'
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [4]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=256)

In [5]:
import os
import threading

In [6]:
!pip install googletrans==4.0.0rc1 diffusers transformers accelerate xformers pyngrok

  Preparing metadata (setup.py) ... done
  Obtaining dependency information for diffusers from https://files.pythonhosted.org/packages/aa/0b/af1dd4b355accf28fa413d93c3b7df2c25922ebc3435eecc56818a3f8c1e/diffusers-0.25.0-py3-none-any.whl.metadata
  Obtaining dependency information for xformers from https://files.pythonhosted.org/packages/f4/89/ce8e936d3e64b3b565c16312dd6446d54f6e485f864130702c6b3b3cbe7c/xformers-0.0.23.post1-cp310-cp310-manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pyngrok from https://files.pythonhosted.org/packages/77/d7/36e6b44465e994e7dc4a434f69f0db66f861e5c303ef36dbbc4f49224ebe/pyngrok-7.0.5-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 36.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.0 MB/s eta 0:00:00
   

In [7]:
# -*- coding: utf-8 -*-

In [9]:
from flask import Flask, render_template, jsonify, request
from pyngrok import ngrok
import traceback
import base64

os.environ["FLASK_ENV"] = "development"
app = Flask(__name__)
port = 5000

app.config['JSON_AS_ASCII'] = False
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = True

ngrok.set_auth_token("2aOzkYCgk7Iz61PVhVcZIGWwwLG_6EQ8cn9Ln5zoJfsroi9Cr")
public_url = ngrok.connect(port).public_url
print(" * ngrok \"{}\" - \"http://127.0.0.1:{}\"".format(public_url,port))

app.config["BASE_URL"] = public_url

@app.route('/chatbot', methods=['POST'])
def chatbot():
    try:
        content = request.json
        prompt = content['prompt']

        result = pipe(prompt)
        list_result = []
        for tmp in result:
            utf8_str = tmp['generated_text'].encode('utf-8')

            # Mã hóa Base64
            base64_encoded = base64.b64encode(utf8_str)

            # Chuyển bytes thành chuỗi
            base64_encoded_text = base64_encoded.decode("utf-8")
            
            list_result.append(base64_encoded_text)
        return jsonify(list_result)

    except Exception as e:
        return jsonify({'error': str(e)})

threading.Thread(target = app.run, kwargs={"use_reloader": False}).start()

 * ngrok "https://018d-34-105-23-20.ngrok-free.app" - "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
